# Decorators continuation

https://realpython.com/primer-on-python-decorators/#a-few-real-world-examples

## A Few Real World Examples

### Template for building more complex decorators

```python
def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```  


### Template to pass an argument to a decorator
```python
def repeat(num_times):
    def decorator_repeat(func):
        ...  # Create and return a wrapper function
    return decorator_repeat
```

In [1]:
import functools
import time

### Example: Timing Functions

In [2]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum_to_waste_time = sum([i**2 for i in range(1000)])
    return sum_to_waste_time

# Execution with print() added
print(waste_some_time(1))
print(waste_some_time(999))

Finished 'waste_some_time' in 0.0003 secs
332833500
Finished 'waste_some_time' in 0.2667 secs
332833500


### Debugging code

**Code in `decorators.py` to create an own module**  
`##################################`

In [3]:
# Debug function.
# The numbered lines of the code do:

# 1. Create a list of the positional arguments. 
#   Use 'repr()' to get a nice string representing each argument.

# 2. Create a list of the keyword arguments. 
#   The 'f-string' formats each argument as key=value where the !r 
#   specifier means that repr() is used to represent the value.

# 3. The lists of positional and keyword arguments is joined with 
#   each argument separated by a comma.

# 4. The return value is printed after the function is executed.

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug


`##################################`  
**End of `decorators.py`**

### Example: Decorator applied to debug a simple function

In [7]:

@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

# print() added
print(make_greeting("Benjamin"))
print('-------')
print(make_greeting("Richard", age=112))

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'
Howdy Benjamin!
-------
Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'
Whoa Richard! 112 already, you are growing up!


### Example: Aproximation of constant e

Application of a decorator to a previously defined function  

In [5]:
import math
# from decorators import debug  # Decorator above

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))  

# approximate_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

### Example: Slowing Down Code

In [6]:
def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

countdown(3)

3
2
1
Liftoff!


### Example: Registering plugins

Here there is no wrapper function.  

Note that you do not have to write an inner function or use `@functools.wraps` in this 
example because you are returning the original function unmodified.

In [17]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

# PLUGINS is populated when the plugins register themselves. 
# This makes it trivial to add a new plugin: just define the 
# function and decorate it with @register.

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

print(PLUGINS)
randomly_greet("Alice")

{'say_hello': <function say_hello at 0x7f096819d9d0>, 'be_awesome': <function be_awesome at 0x7f096819da60>}
Using 'be_awesome'


'Yo Alice, together we are the awesomest!'